Matthieu Antony, in AEDES, developed an algorithm to orient the program's supervision. This algorithm works in six steps :

1. Computation of aggregate level of payment diminution for each indicator
2. Determination of a subset of N indicators that cause most of payment diminutions
3. Computation of a weighted correction rate of these indicators for each facility, using a priori weights displaying the qualitative interest of each activity
4. Computation of the weighted correction amount for these indicators for each facility
5. Classification of facilities in three levels of risks, based on their weighted correction rates and correction amounts, and predefined threshold
6. Monthly sampling of each facilities, with different sampling probabilities depending on facilities class

Let's try and replicate his results

In [4]:
import pandas as pd
import pickle
import numpy as np

# For warnings
import warnings
warnings.filterwarnings('ignore')

store = pd.HDFStore('../../data/processed/orbf_benin.h5')
tarifs = store['tarifs']
data = store['data']
store.close()

First let's subset the data to the data reported by the WB Zones de Santé for the year 2016 :

In [5]:
bm_zones =['OUIDAH-KPOMASSE-TORI-BOSSITO', 'BANIKOARA', 'LOKOSSA-ATHIEME' , 'ADJOHOUN-BONOU-DANGBO',       'KOUANDE-OUASSA-PEHUNCO-KEROU','COVE-ZANGNANADO-OUINHI', 'PORTONOVO-AGUEGUES-SEME-PODJI', 'BOHICON-ZAKPOTA-ZOGBODOMEY']

data = data[data.geozone_name.isin(bm_zones)]
data = data[data.period.apply(str).str[0:4] == '2016']


data = data.set_index('indicator_label')

I then want to reproduce the first table in the presentation

In [23]:
## Making payment claimed and verified

data['claimed_payment'] = list(data.indicator_claimed_value * tarifs)
data['verified_payment'] = list(data.indicator_verified_value * tarifs)

## Getting total amount of money the program got back 
def get_ecarts(data):
    return sum(data['indicator_claimed_value'] - data['indicator_verified_value']) / sum(data['indicator_claimed_value'])

def get_revenu_gagne(data):
    return sum(data['claimed_payment'])

def get_volume_financier_recupere(data) :
    return sum(data['verified_payment'] - data['claimed_payment'])

def make_first_table(data):
    col1 = data.groupby(level = 0).apply(get_ecarts)
    col2 = data.groupby(level = 0).apply(get_revenu_gagne)
    col3 = data.groupby(level = 0).apply(get_volume_financier_recupere)
    col4 = col3 / get_volume_financier_recupere(data)
    col5 = col4.cumsum()
    return col1
    
    output = col1.to_frame().merge(col2.to_frame()).merge(col3.to_frame).merge(col4.to_frame).merge(col5.to_frame)
    
    output = output.sort_values('% Volume Financier Récupéré')
    return output


make_first_table(data)

indicator_label
Accouchement eutocique assiste                                                                                         0.023308
Accouchement eutocique assiste femme indigente (prime additionnelle)                                                   0.335878
Accouchement: reference d'urgence pour accouchement                                                                    0.233482
CPN 1 standard (y compris VAT, MILD et supplémentation en micronutriments)                                             0.070470
CPN 1 standard (y compris VAT, MILD et supplémentation en micronutriments) pour les Indigentes (prime additionelle)    0.682927
Clients sous traitement ARV suivi pendant les 6 premiers mois                                                          0.418546
Consultation postnatale 1 (du 7ème au 10ème jour)                                                                      0.060664
Consultation prenatale quatrieme visite standard                                        

In [37]:
def make_first_table(data):
    col1 = data.groupby(level = 0).apply(get_ecarts)
    col2 = data.groupby(level = 0).apply(get_revenu_gagne)
    col3 = data.groupby(level = 0).apply(get_volume_financier_recupere)
    col4 = col3 / get_volume_financier_recupere(data)
    col5 = col4.cumsum()
    return col2

In [38]:
a = make_first_table(data)

In [32]:
113036000/ 0.07

1614799999.9999998

In [43]:
data.indicator_tarif['Diagnostic et traitement des cas de paludisme simple chez les enfants'].unique()

array([ 330.])

In [39]:
b = a * tarifs

In [48]:
1.902075e+10 / 169350000 

112.31620903454385

In [49]:
3000.0 / 112.31620903454385

26.7103032214818

In [44]:
tarifs

Diagnostic et traitement des cas de paludisme simple chez les enfants                                                    330.0
Diagnostic et traitement des cas de paludisme sévère chez les enfants                                                   5890.0
Diagnostic et traitement des cas de paludisme simple chez les femmes enceintes                                           655.0
Diagnostic et traitement des cas de paludisme grave chez les femmes enceintes                                           5365.0
Femmes enceintes dépistées séropositive et mise sous traitement ARV(tri prophylaxie/trithérapie)                       15750.0
Clients sous traitement ARV suivi pendant les 6 premiers mois                                                           8750.0
Enfants éligibles au traitement ARV et qui ont été initié au traitement ARV au cours du mois                           19250.0
Depistage des cas TBC positifs                                                                                 

In [51]:
b.abs()

Accouchement eutocique assiste                                                                                         1.069013e+11
Accouchement eutocique assiste femme indigente (prime additionnelle)                                                   2.301189e+11
Accouchement: reference d'urgence pour accouchement                                                                    6.512990e+11
CPN 1 standard (y compris VAT, MILD et supplémentation en micronutriments)                                             4.259325e+10
CPN 1 standard (y compris VAT, MILD et supplémentation en micronutriments) pour les Indigentes (prime additionelle)    3.535938e+11
Clients sous traitement ARV suivi pendant les 6 premiers mois                                                          3.054844e+10
Consultation postnatale 1 (du 7ème au 10ème jour)                                                                      5.812076e+11
Consultation prenatale quatrieme visite standard                            